# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ
# génération d'un fichier light de 1000 lignes pour traitements plus rapides.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv"
df_emails.to_csv(file_name, encoding='utf-8', index=False)

Mounted at /content/drive
(405968, 6)
(227032, 5)


# Import - Data Preprocessing structures

Mount Gdrive files

In [40]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import Spacy

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
# ligne ci-dessous a réexecuter ssi reinit du notebook
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'


In [ ]:
import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df_emails = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv", nrows=1000)
df_emails.head()

Overview dataset

In [ ]:
# nbre d'expéditeurs distincts
len(df_emails['from'].unique())

In [ ]:
# nbre de destinataires distincts
len(df_emails['to'].unique())

Rajout colonnes pour traitements ultérieurs

In [ ]:
import numpy as np
df_emails = df_emails.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]

df_emails['cc'] = ''                        # pour identfication des personnes en copie
df_emails['NER_header'] = ''                # Spacy NER
df_emails['NER_body'] = ''                  # Spacy NER
df_emails['body_clean'] = ''                # body sans caracteres parasites
df_emails['body_dict'] = ''                 # body tokenisé
df_emails['summary_TFIDF'] = ''             # extractive summary by TF-IDF
df_emails['summary_TFIDF_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_spacy'] = ''             # extractive summary by Spacy
df_emails['summary_spacy_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_abstractive'] = ''       # abstractive summary by HugginFace/BERT
df_emails['summary_abstractive_sim'] = ''   # calcul de similarité par rapport à body_clean
df_emails['best_sim'] = ''                  # meilleure taux de similarité par rapport à body_clean
df_emails.info()


In [ ]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

# NER with Spacy

In [48]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)


deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

In [49]:
df_emails[['header','NER_header','NER_body']][0:40]

,header,NER_header,NER_body
0,NaN,(),()
1,Re:,(),"((Austin),)"
2,Re: test,(),()
3,NaN,(),"((Randy), (Patti, S))"
4,Re: Hello,(),"((Tuesday), (11:45))"
5,Re: Hello,(),"((next, Tuesday), (Thursday))"
6,NaN,(),"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"((PRC),)","((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,(),"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"((FW, :), (Collar))","((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [50]:
id = 12
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")

----- eMail  12  -----


In [51]:
df_emails[['header','body','body_dict','summary_TFIDF','NER_body']][0:30]

,header,body,body_dict,summary_TFIDF,NER_body
0,NaN,"['', 'Here is our forecast', '', ' ']",,,()
1,Re:,"['', 'Traveling to have a business meeting tak...",,,"((Austin),)"
2,Re: test,"['', 'test successful. way to go!!!']",,,()
3,NaN,"['', 'Randy,', '', ' Can you send me a schedul...",,,"((Randy), (Patti, S))"
4,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]",,,"((Tuesday), (11:45))"
5,Re: Hello,"['', 'Greg,', '', ' How about either next Tues...",,,"((next, Tuesday), (Thursday))"
6,NaN,"['', 'Please cc the following distribution lis...",,,"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"['', 'any morning between 10 and 11:30']",,,"((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,"['', '1. login: pallen pw: ke9davis', '', "" I...",,,"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"['', '---------------------- Forwarded by Phil...",,,"((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [52]:
# import spacy
# from spacy import displacy
# nlp = spacy.load('en_core_web_sm')
# spacy_stopwords = nlp.Defaults.stop_words
# stopwords = spacy_stopwords
# print(spacy_stopwords)

# Data cleaning

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \“‘,]
#  print(“Data cleaning --\n”)
#  print(“Avant :\n”,type(tokens),” “,tokens)
   r = re.compile(r"[\\\''\'\"\[\]]+")
   tok=r.sub('', tokens)
   r = re.compile(r"([\(\)]{1,}|,{2,})")
   tok2=r.sub(' ', tok)
   r = re.compile(r"((\.\s?,)+|(, ,)|(^,{1,}))")
   tok3=r.sub('', tok2)
   r = re.compile(r"\s{2,}")
   texte=r.sub(' ', tok3)
#  texte = sent_tokenize(texte)  # découpe en phrases stockées dans une liste
#  print(“Après sent_tokenize :\n”,type(texte),” “,texte)
   return texte

In [54]:
def remove_stop_words(text,stopwords):
  lst=[]
#  for token in str(text).split():
  for token in text.split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
#    else:
#      print("suppression du stopword : ",token)
  # Join items in the list
#   print("Original text before removing stopwords  :   \n",text)
  result = ' '.join(lst)
#   print("Text after removing stopwords  :   \n",result)
  return result

In [55]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [56]:
df_emails.shape[0]

1000

# Summary methodS

In [57]:
import pandas as pd
import numpy as np


def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize_TFIDF(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    if len(corpus) <= n_sentences:
      print("Longeur corpus inférieure au nombre minimal de phrases retenu pour le résumé")
      return texte
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
#    print("[",i,"] :",summary)
    # transformation finale en chaine de caracteres
    texte = ''.join(summary)
    return texte

def summarize_spacy (text,ratio):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  nlp = spacy.load('en_core_web_sm')
  doc= nlp(text)
  tokens=[token.text for token in doc]
#  print("Tokens : \n",tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
#  print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
#  print("\n Normalized word_frequencies : \n",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
#  print(sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier % (ratio) du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*ratio)
  select_length
  summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
  final_summary=[word.text for word in summary]
  summary=''.join(final_summary)
  return summary



In [58]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
def summarize_abstractive (text, min_length, max_length):
  print("paramètres :", min_length, max_length)
  print("texte : ",len(text),text)
  from transformers import pipeline
#  print("Avant : ",text)
  # use the BART model, which is trained on the CNN/Daily Mail News Dataset, you can directly use the default parameters
  # bart-large-cnn model for summarization
  summarizer = pipeline("summarization")
  summarized = summarizer(text, min_length, max_length)
  summary=' '.join([str(i) for i in summarized])
#  print("Après : ",summary)
  return summary

from transformers import pipeline
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#def summarize_T5 (text, min_length, max_length):
def summarize_T5 (text):
  summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
  summary_text = summarizer(text, max_length=100, min_length=5, do_sample=False)[0]['summary_text']
  return summary_text
  

# Test Unitaire

In [63]:
id = 44
df_emails['body_clean'][id] = data_clean(df_emails['body'].iloc[id])

text = df_emails['body_clean'][id]
print("Original text : ")
print(df_emails['body'][id])
print("\nCleaned text : ",type(text),len(text)," car. : ")
print(text,"\n")
# Extractive summary using TF-IDF
# ------------------------------------------------------------------------
# suppression de stopwords pour TFIDF
text = remove_stop_words(text,stopwords)

# Dictionnarize the text to be used with TF-IDF
print("Extractive summary using TF-IDF")
max_sent = 3
df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
if len(df_emails['body_dict'][id]) > max_sent:
  text = df_emails['body_dict'][id]
  # on affiche phrase par phrase le contenu de chaque mail
  for cle, valeur in text.items():
    print("Ligne ", cle, " : ", valeur)
  # Identification des max_sent les plus significatives 
  df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
  ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_TFIDF'][id])
else:
  print("ID ",id," contenu trop court. Reprise de l'original pour TFIDF")
  df_emails['summary_TFIDF'][id] = df_emails['body_clean'][id]

# ------------------------------------------------------------------------
# Extractive summary using Spacy
print("\nExtractive summary using Spacy")
ratio = 0.30
text = df_emails['body_clean'].iloc[id]
df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
print("Résumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
print(df_emails['summary_spacy'][id])

# ------------------------------------------------------------------------
# Extractive summary using Transformers
# BART model
print("\nExtractive summary using Transformers / BART")
text = df_emails['body_clean'].iloc[id]
try:
  df_emails['summary_abstractive'][id] = summarize_abstractive (text, min_length=30, max_length=100)
  ratio = len(df_emails['summary_abstractive'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Transformers BART  - Len ",len(df_emails['summary_abstractive'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_abstractive'][id])
except:
  print("Pb extractive summary on Id ",id)
  df_emails['summary_abstractive'][id] = "Erreur"

# # T5 model



Original text : 
['', 'Jeff,', '', ' I need to see the site plan for Burnet.  Remember I must get written ', 'approval from Brenda Key Stone before I can sell this property and she has ', 'concerns about the way the property will be subdivided.    I would also like ', 'to review the closing statements as soon as possible.', '', 'Phillip']

Cleaned text :  <class 'str'> 293  car. : 
 Jeff , I need to see the site plan for Burnet. Remember I must get written , approval from Brenda Key Stone before I can sell this property and she has , concerns about the way the property will be subdivided. I would also like , to review the closing statements as soon as possible , Phillip 

Extractive summary using TF-IDF
ID  44  contenu trop court. Reprise de l'original pour TFIDF

Extractive summary using Spacy


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Résumé Spacy - Len  0 Ratio :  0 %


Extractive summary using Transformers / BART
paramètres : 30 100
texte :  293  Jeff , I need to see the site plan for Burnet. Remember I must get written , approval from Brenda Key Stone before I can sell this property and she has , concerns about the way the property will be subdivided. I would also like , to review the closing statements as soon as possible , Phillip


Ignoring args : (30, 100)
Your max_length is set to 142, but you input_length is only 64. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)



Résumé Transformers BART  - Len  300  Ratio :  102 %
{'summary_text': ' Jeff , I need to see the site plan for Burnet. Remember I must get written approval from Brenda Key Stone before I can sell this property . Brenda has concerns about the way the property will be subdivided. I would also like to review the closing statements as soon as possible .'}


In [61]:
df_emails['T5'] = ''

In [62]:
print("\nExtractive summary using Transformers / T5")
text = df_emails['body_clean'].iloc[id]
text = """One month after the United States began what has become a troubled rollout of a national COVID vaccination campaign, the effort is finally gathering real steam.
Close to a million doses -- over 951,000, to be more exact -- made their way into the arms of Americans in the past 24 hours, the U.S. Centers for Disease Control and Prevention reported Wednesday. That's the largest number of shots given in one day since the rollout began and a big jump from the previous day, when just under 340,000 doses were given, CBS News reported.
That number is likely to jump quickly after the federal government on Tuesday gave states the OK to vaccinate anyone over 65 and said it"""
print(text)
try:
  df_emails['summary_T5'][id] = summarize_T5 (text)
  ratio = len(df_emails['summary_T5'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Transformers T5  - Len ",len(df_emails['summary_T5'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_T5'][id])
except:
  print("Pb extractive summary on Id ",id)
  df_emails['summary_T5'][id] = "Erreur"


Extractive summary using Transformers / T5
One month after the United States began what has become a troubled rollout of a national COVID vaccination campaign, the effort is finally gathering real steam.
Close to a million doses -- over 951,000, to be more exact -- made their way into the arms of Americans in the past 24 hours, the U.S. Centers for Disease Control and Prevention reported Wednesday. That's the largest number of shots given in one day since the rollout began and a big jump from the previous day, when just under 340,000 doses were given, CBS News reported.
That number is likely to jump quickly after the federal government on Tuesday gave states the OK to vaccinate anyone over 65 and said it


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid th

Pb extractive summary on Id  44


KeyError: ignored

In [ ]:
text = df_emails['body_clean'].iloc[id]
text

In [ ]:
text
summary_text = summarizer(text, max_length=100, min_length=5, do_sample=False)[0]['summary_text']

In [ ]:
id=20
id, len(df_emails['body_clean'][id]),(df_emails['body_clean'][id])


In [ ]:
id, (df_emails['body_dict'][id])

In [ ]:
id, len(df_emails['body_dict'][id])

In [ ]:
id, df_emails['summary_TFIDF'][id]

# Boucle de traitement

In [ ]:
deb = 0
fin = df_emails.shape[0]
print("Traitement en cours pour ",fin, "records")
fin = 100
for id in range(deb,fin):
  print("\n id : ",id)
  print('--------------------')
  # coller quand OK pour les tests unitaires
  df_emails['body_clean'][id] = data_clean(str(df_emails['body'].iloc[id]))
  text = df_emails['body_clean'][id]
  print("Cleaned text : ",len(str(text))," car.\n")
  print(text)
  # Extractive summary using TF-IDF
  # ------------------------------------------------------------------------
  # suppression de stopwords pour TFIDF
  text = remove_stop_words(text,stopwords)

  # Dictionnarize the text to be used with TF-IDF
  df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
  max_sent = 3 
  if len(df_emails['body_dict'][id]) > max_sent:
    text = df_emails['body_dict'][id]
    # on affiche phrase par phrase le contenu de chaque mail
    for cle, valeur in text.items():
      print("Ligne ", cle, " : ", valeur)
    # Identification des n_sent les plus significatives

    df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
    ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
    print(df_emails['summary_TFIDF'][id])
  else:
    print("ID ",id," contenu trop court. Reprise de l'original pour TFIDF")
    df_emails['summary_TFIDF'][id] = df_emails['body_clean'][id]

# ------------------------------------------------------------------------
  # Extractive summary using Spacy
  ratio = 0.20
  text = df_emails['body_clean'].iloc[id]
  df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
  ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_spacy'][id])

# ------------------------------------------------------------------------
# Extractive summary using Transformers
  try:
    text = df_emails['body_clean'].iloc[id]
    df_emails['summary_abstractive'][id] = summarize_abstractive (text, min_length=30, max_length=100)
    ratio = len(df_emails['summary_abstractive'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé Transformers BERT  - Len ",len(df_emails['summary_abstractive'][id])," Ratio : ",int(ratio*100),"%")
    print(df_emails['summary_abstractive'][id])
  except:
    print("Pb extractive summary on Id ",id)
    df_emails['summary_abstractive'][id] = "Erreur"


In [ ]:
# doc1.similarity(doc2)
!python -m spacy download en_core_web_lg

In [ ]:
df_emails.head(20)

In [ ]:

nlp = spacy.load('en_core_web_lg')
id=11
doc1 = nlp(df_emails['body_clean'][id])
doc2 = nlp(df_emails['summary_spacy'][id])

# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

# SPACY - Summary (Abstractive / Word Embedding)

In [ ]:
#!pip install spacy-transformers

In [ ]:
# https://spacy.io/usage/embeddings-transformers
# https://spacy.io/api/transformer




In [ ]:
# https://manusidtech.com/index.php/2021/08/14/complete-nlp-web-app-using-spacy-and-hugging-face-transformers/
#!pip install datasets transformers[sentencepiece]
#text = str(df_emails['body'][32])
#text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
#print(text)

#print(summarize_abstractive (text))
#print(summarize_spacy (text))
#print()

A voir

In [ ]:
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
# https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510
# https://www.analyticsvidhya.com/blog/2021/10/text-summarization-using-the-conventional-hugging-face-transformer-and-cosine-similarity/


In [ ]:
#!python -m spacy download en_core_web_lg

# Calcul similarités des résumés

In [ ]:
spacy.load('en_core_web_lg')

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
deb = 1
fin = df_emails.shape[0]
#fin = 21
print("Traitement en cours pour ",fin, "records")

for id in range(deb,fin):
  print("ID:",id)
  # Calcul de similarité avec texte initial nettoyé
  doc1 = nlp(df_emails['body_clean'][id])
  max_sim = 0
  doc2 = nlp(df_emails['summary_TFIDF'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_TFIDF_sim'][id] = sim
    max_sim = sim

  doc2 = nlp(df_emails['summary_spacy'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_spacy_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  doc2 = nlp(df_emails['summary_abstractive'][id])
  if doc2 != "Erreur":
    sim = doc1.similarity(doc2)
    df_emails['summary_abstractive_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  print("Meilleur score : ",max_sim)
  df_emails['best_sim'][id] = max_sim


In [ ]:
df_emails


# Sauvegarde du fichier final en .csv et .xls

In [ ]:
# SAUVEGARDE en csv du résultat.
fin = df_emails.shape[0]
fin = 200
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output.csv"
df_emails[deb:fin].to_csv(file_name, encoding='utf-8', index=False)
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output.xls"
df_emails[deb:fin].to_excel(file_name, encoding='utf-8', index=False)



In [ ]:
# to gsheet
import gspread
# from gspread_dataframe import get_as_dataframe, set_with_dataframe
# set_with_dataframe(worksheet, df_emails)
sh = gc.create("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output")

# Archived